# Labelme转COCO

同济子豪兄 呕心沥血写成

兼容目标检测、图像分割、关键点检测三种标注

2023-3-9

2023-3-10

2023-4-15

## 导入工具包

In [1]:
import os
import json

import numpy as np

## 删除系统自动生成的多余文件

建议在 Linux 系统中运行爬虫、划分训练集测试集代码

### 查看待删除的多余文件

In [2]:
!find . -iname '__MACOSX'

In [3]:
!find . -iname '.DS_Store'

In [4]:
!find . -iname '.ipynb_checkpoints'

./labelme_jsons_dataset/train_labelme_jsons/.ipynb_checkpoints
./labelme_jsons_dataset/.ipynb_checkpoints
./.ipynb_checkpoints


### 删除多余文件

In [5]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [6]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [7]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [8]:
!find . -iname '__MACOSX'

In [9]:
!find . -iname '.DS_Store'

In [10]:
!find . -iname '.ipynb_checkpoints'

## 进入某一个labelme标注文件的目录

In [11]:
labelme_folder = 'labelme_jsons_dataset/train_labelme_jsons'

In [12]:
os.chdir(labelme_folder)

In [13]:
os.listdir()

['DSC_0284.json',
 'DSC_0281.json',
 'DSC_0282.json',
 'DSC_0274.json',
 'DSC_0260.json',
 'DSC_0280.json',
 'DSC_0285.json',
 'DSC_0265.json',
 'DSC_0219.json',
 'DSC_0289.json',
 'DSC_0209.json',
 'DSC_0301.json',
 'DSC_0283.json',
 'DSC_0245.json',
 'DSC_0278.json',
 'DSC_0236.json']

## 示例-导入一个labelme格式的json标注文件

In [14]:
# with open('1.json', 'r', encoding='utf-8') as f:
#     labelme = json.load(f)

In [15]:
# labelme['shapes']

In [16]:
# labelme.keys()

In [17]:
# labelme['version']

In [18]:
# labelme['flags']

In [19]:
# 图像文件名
# labelme['imagePath']

In [20]:
# labelme['imageData']

In [21]:
# 图像高度
# labelme['imageHeight']

In [22]:
# 图像宽度
# labelme['imageWidth']

## 创建coco格式的字典

In [23]:
coco = {}

## info

In [24]:
# coco['info'] = {}
# coco['info']['description'] = 'Labelme2coco keypoint format script from Zihao'
# coco['info']['year'] = 2023
# coco['info']['date_created'] = '2023/03/09'

## categories

supercategory：框的类别（一般只有一个类别）

keypoints：节点类别排序，后续的关键点坐标展示顺序

skeleton：可有可无，可视化时需要

In [25]:
class_list = {
    'supercategory': 'sjb_rect',
    'id': 1,
    'name': 'sjb_rect',
    'keypoints': ['angle_30', 'angle_60', 'angle_90'], # 大小写敏感
    'skeleton':[[0,1], [0,2], [1,2]]
}

In [26]:
coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

## 函数-处理单个labelme标注json文件

In [27]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: # 遍历该json文件中的所有标注

        if each_ann['shape_type'] == 'rectangle': # 筛选出个体框

            # 个体框元数据
            bbox_dict = {}
            bbox_dict['category_id'] = 1
            bbox_dict['segmentation'] = []
            
            bbox_dict['iscrowd'] = 0
            bbox_dict['segmentation'] = []
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            # print(ANN_ID)
            ANN_ID += 1

            # 获取个体框坐标
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            bbox_dict['area'] = bbox_w * bbox_h
            
            # 筛选出分割多段线
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'polygon': # 筛选出分割多段线标注
                    # 第一个点的坐标
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                        # bbox_dict['segmentation'] = each_ann['points']

            # 筛选出该个体框中的所有关键点
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点横纵坐标
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_left_top_x) & (x<bbox_right_bottom_x) & (y<bbox_right_bottom_y) & (y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
                        
            bbox_dict['num_keypoints'] = len(bbox_keypoints_dict)
            # print(bbox_keypoints_dict)

            # 把关键点按照类别顺序排好
            bbox_dict['keypoints'] = []
            for each_class in class_list['keypoints']:
                if each_class in bbox_keypoints_dict:
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][0])
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][1])
                    bbox_dict['keypoints'].append(2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

## 测试一下函数的效果，处理单个labelme格式的json标注文件

In [28]:
with open('DSC_0284.json', 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [29]:
process_single_json(labelme)

[{'category_id': 1,
  'segmentation': [],
  'iscrowd': 0,
  'image_id': 1,
  'id': 0,
  'bbox': [852, 122, 3392, 1552],
  'area': 5264384,
  'num_keypoints': 3,
  'keypoints': [922, 1396, 2, 4218, 1600, 2, 3450, 156, 2]},
 {'category_id': 1,
  'segmentation': [],
  'iscrowd': 0,
  'image_id': 1,
  'id': 1,
  'bbox': [1072, 2006, 3324, 1592],
  'area': 5291808,
  'num_keypoints': 3,
  'keypoints': [4354, 2472, 2, 1146, 2060, 2, 1758, 3568, 2]}]

## images和annotations

In [30]:
IMG_ID = 0
ANN_ID = 0

In [31]:
# 遍历所有 labelme 格式的 json 文件
for labelme_json in os.listdir(): 
    
    if labelme_json.split('.')[-1] == 'json':
        
        with open(labelme_json, 'r', encoding='utf-8') as f:
            
            labelme = json.load(f)
            
            ## 提取图像元数据
            img_dict = {}
            img_dict['file_name'] = labelme['imagePath']
            img_dict['height'] = labelme['imageHeight']
            img_dict['width'] = labelme['imageWidth']
            img_dict['id'] = IMG_ID
            coco['images'].append(img_dict)
            
            ## 提取框和关键点信息
            coco_annotations = process_single_json(labelme, image_id=IMG_ID)
            coco['annotations'] += coco_annotations
            
            IMG_ID += 1
            
            print(labelme_json, '已处理完毕')

    else:
        pass

DSC_0284.json 已处理完毕
DSC_0281.json 已处理完毕
DSC_0282.json 已处理完毕
DSC_0274.json 已处理完毕
DSC_0260.json 已处理完毕
DSC_0280.json 已处理完毕
DSC_0285.json 已处理完毕
DSC_0265.json 已处理完毕
DSC_0219.json 已处理完毕
DSC_0289.json 已处理完毕
DSC_0209.json 已处理完毕
DSC_0301.json 已处理完毕
DSC_0283.json 已处理完毕
DSC_0245.json 已处理完毕
DSC_0278.json 已处理完毕
DSC_0236.json 已处理完毕


## 保存为MS COCO格式的json标注文件

In [32]:
coco_path = '../train_coco.json'
# coco_path = '../val_coco.json'

In [33]:
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)

In [34]:
coco

{'categories': [{'supercategory': 'sjb_rect',
   'id': 1,
   'name': 'sjb_rect',
   'keypoints': ['angle_30', 'angle_60', 'angle_90'],
   'skeleton': [[0, 1], [0, 2], [1, 2]]}],
 'images': [{'file_name': 'DSC_0284.jpg',
   'height': 3712,
   'width': 5568,
   'id': 0},
  {'file_name': 'DSC_0281.jpg', 'height': 3712, 'width': 5568, 'id': 1},
  {'file_name': 'DSC_0282.jpg', 'height': 3712, 'width': 5568, 'id': 2},
  {'file_name': 'DSC_0274.jpg', 'height': 5568, 'width': 3128, 'id': 3},
  {'file_name': 'DSC_0260.jpg', 'height': 5568, 'width': 3128, 'id': 4},
  {'file_name': 'DSC_0280.jpg', 'height': 3712, 'width': 5568, 'id': 5},
  {'file_name': 'DSC_0285.jpg', 'height': 3712, 'width': 5568, 'id': 6},
  {'file_name': 'DSC_0265.jpg', 'height': 5568, 'width': 3128, 'id': 7},
  {'file_name': 'DSC_0219.jpg', 'height': 5568, 'width': 3712, 'id': 8},
  {'file_name': 'DSC_0289.jpg', 'height': 3128, 'width': 5568, 'id': 9},
  {'file_name': 'DSC_0209.jpg', 'height': 3712, 'width': 5568, 'id': 10},

## 验证MS COCO格式的标注

In [36]:
from pycocotools.coco import COCO

my_coco = COCO(coco_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## 分别运行训练集和测试集，`train_coco.json`和`val_coco.json`即为COCO格式的json文件